In [ ]:
import os
import requests
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.affinity import scale, translate
import matplotlib.patheffects as pe

# Directory to store downloaded data
local_data_dir = "./data/"
os.makedirs(local_data_dir, exist_ok=True)

# eGRID Data URLs
egrid_urls = {
    2019: {
        "egrid_xlsx": 'https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx',
        "subregions_kmz": 'https://www.epa.gov/sites/default/files/2021-02/egrid2019_subregions.kmz',
    },
    2020: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2022-09/eGRID2020_Data_v2.xlsx',
        "subregions_kmz": 'https://www.epa.gov/system/files/other-files/2022-01/egrid2020_subregions.kmz',
    },
    2021: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2023-01/eGRID2021_data.xlsx',
        "subregions_shp": 'https://gaftp.epa.gov/EPADataCommons/ORD/egrid2021/eGRID2021%20Subregions%20Shapefile.zip',
    },
    2022: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2023-01/egrid2022_data.xlsx',
        "subregions_kmz": 'https://www.epa.gov/system/files/other-files/2023-01/egrid2022_subregions.kmz',
    },
}

# Conversion factor from pounds to kilograms
LBS_PER_KG = 0.45359237

# Function to download files
def download_file(url, local_path):
    if not os.path.exists(local_path):
        print(f"Downloading from {url}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded to {local_path}.")
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
    else:
        print(f"{local_path} already downloaded.")

# Function to extract data for a given year
def process_year(year, urls):
    print(f"\nProcessing data for year {year}...")

    # Paths for Excel and subregions files
    egrid_xlsx_url = urls['egrid_xlsx']
    egrid_xlsx_path = os.path.join(local_data_dir, f"egrid{year}_data.xlsx")

    # Determine the subregions file type and path
    if 'subregions_kml' in urls:
        subregions_url = urls['subregions_kml']
        subregions_ext = 'kml'
    elif 'subregions_kmz' in urls:
        subregions_url = urls['subregions_kmz']
        subregions_ext = 'kmz'
    elif 'subregions_shp' in urls:
        subregions_url = urls['subregions_shp']
        subregions_ext = 'zip'
    else:
        print(f"No subregions file provided for year {year}. Skipping...")
        return None

    subregions_path = os.path.join(local_data_dir, f"egrid{year}_subregions.{subregions_ext}")

    # Download files
    download_file(egrid_xlsx_url, egrid_xlsx_path)
    download_file(subregions_url, subregions_path)

    # Load carbon intensity data
    try:
        sheet_name = f'SRL{str(year)[-2:]}'
        egrid_intensity_df = pd.read_excel(egrid_xlsx_path, sheet_name=sheet_name)

        # Use the relevant column for CO2 intensity
        co2_column_name = [col for col in egrid_intensity_df.columns if 'CO2 total output emission rate (lb/MWh)' in col]
        if co2_column_name:
            co2_column_name = co2_column_name[0]
        else:
            raise ValueError("CO2 emission rate column not found.")

        # Extract subregion acronym and carbon intensity
        region_intensity = egrid_intensity_df[['eGRID subregion acronym', co2_column_name]].copy()
        region_intensity.columns = ['SUBRGN', 'SRC2ERTA']

        # Convert to numeric and handle NaNs
        region_intensity['SRC2ERTA'] = pd.to_numeric(region_intensity['SRC2ERTA'], errors='coerce')
        region_intensity = region_intensity.dropna(subset=['SRC2ERTA'])

        # Convert lbs to kg
        region_intensity['kg_per_mwh'] = region_intensity['SRC2ERTA'] * LBS_PER_KG
        region_intensity = region_intensity[['SUBRGN', 'kg_per_mwh']]
        region_intensity['SUBRGN'] = region_intensity['SUBRGN'].astype(str)

        print(f"Extracted intensity data for {year}:")
        print(region_intensity.head())
        print(f"{year} - Available subregions in intensity data:", region_intensity['SUBRGN'].unique())

    except Exception as e:
        print(f"Failed to read eGRID data for {year}: {e}")
        return None

    # Load subregions into GeoDataFrame
    try:
        if subregions_ext == 'kmz':
            with ZipFile(subregions_path, 'r') as kmz:
                kml_filename = [name for name in kmz.namelist() if name.endswith('.kml')][0]
                kmz.extract(kml_filename, local_data_dir)
                kml_path = os.path.join(local_data_dir, kml_filename)
                gdf = gpd.read_file(kml_path, driver='KML')
                os.remove(kml_path)
        elif subregions_ext == 'kml':
            gdf = gpd.read_file(subregions_path, driver='KML')
        elif subregions_ext == 'zip':
            # Extract the zip file
            with ZipFile(subregions_path, 'r') as zip_ref:
                zip_ref.extractall(local_data_dir)
                # Find the shapefile (.shp)
                shapefile_name = [name for name in zip_ref.namelist() if name.endswith('.shp')][0]
                shapefile_path = os.path.join(local_data_dir, shapefile_name)
            gdf = gpd.read_file(shapefile_path)
        else:
            print(f"Unsupported subregions file format for year {year}.")
            return None

        # Check for 'SUBRGN' or 'Name' column and rename to 'Name'
        if 'SUBRGN' in gdf.columns:
            gdf = gdf.rename(columns={'SUBRGN': 'Name'})
        elif 'Name' not in gdf.columns:
            raise ValueError("Subregion name column not found in geospatial data.")

        # Ensure geometries are in WGS84 CRS (EPSG:4326)
        if gdf.crs != 'EPSG:4326':
            print(f"Reprojecting geometries to WGS84 for year {year}.")
            gdf = gdf.to_crs(epsg=4326)

        # Merge with carbon intensity data using 'Name' as the identifier
        gdf = gdf.merge(region_intensity, left_on='Name', right_on='SUBRGN', how='left')

    except Exception as e:
        print(f"Failed to read subregions for {year}: {e}")
        return None

    # Adjust regions and return the processed GeoDataFrame
    combined_gdf = adjust_regions(gdf, year)
    return combined_gdf

# Function to adjust regions (without plotting)
def adjust_regions(gdf, year):
    # Define subregions for Alaska and Hawaii
    alaska_subregions = ['AKGD', 'AKMS']
    hawaii_subregions = ['HIMS', 'HIOA', 'HIMS/HIOA']
    puerto_rico_subregions = ['PR', 'PRVI', 'PRMS']

    # Attempt to find Puerto Rico subregions in the data
    pr_subregions = gdf[gdf['Name'].str.contains('PR', case=False)]['Name'].unique()
    if len(pr_subregions) == 0:
        # Try to identify PR subregions differently
        pr_subregions = [name for name in gdf['Name'].unique() if 'PR' in name.upper()]
    print(f"Puerto Rico subregions in data: {pr_subregions}")

    # Separate the regions
    mainland_gdf = gdf[~gdf['Name'].isin(alaska_subregions + hawaii_subregions + list(pr_subregions))]
    alaska_gdf = gdf[gdf['Name'].isin(alaska_subregions)]
    hawaii_gdf = gdf[gdf['Name'].isin(hawaii_subregions)]
    puerto_rico_gdf = gdf[gdf['Name'].isin(pr_subregions)]

    # Alaska: Scale and translate
    if not alaska_gdf.empty:
        alaska_gdf = alaska_gdf.copy()

        # Compute the centroid of Alaska (combined geometries)
        alaska_centroid = alaska_gdf.geometry.unary_union.centroid

        # Desired location for Alaska after translation
        alaska_desired_location = (-120, 27)  # Adjust as needed

        # Apply scaling with the common origin (alaska_centroid)
        alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=0.41, yfact=0.5, origin=alaska_centroid)
        )

        # After scaling, compute the new centroid
        scaled_alaska_centroid = alaska_gdf.geometry.unary_union.centroid

        # Compute translation amounts based on the desired location
        delta_x = alaska_desired_location[0] - scaled_alaska_centroid.x
        delta_y = alaska_desired_location[1] - scaled_alaska_centroid.y

        # Apply translation to each geometry
        alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print("Alaska data is not available.")

    # Hawaii: Scale and translate
    if not hawaii_gdf.empty:
        hawaii_gdf = hawaii_gdf.copy()
        hawaii_centroid = hawaii_gdf.geometry.unary_union.centroid

        # Apply scaling
        hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=1.5, yfact=1.5, origin=hawaii_centroid)
        )

        # Desired location for Hawaii after translation
        hawaii_desired_location = (-105, 22)  # Adjust as needed
        delta_x = hawaii_desired_location[0] - hawaii_centroid.x
        delta_y = hawaii_desired_location[1] - hawaii_centroid.y

        # Apply translation
        hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print("Hawaii data is not available.")

    # Puerto Rico: Scale and translate
    if not puerto_rico_gdf.empty:
        puerto_rico_gdf = puerto_rico_gdf.copy()
        pr_centroid = puerto_rico_gdf.geometry.unary_union.centroid

        # Apply scaling
        puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=1.5, yfact=1.5, origin=pr_centroid)
        )

        # Desired location for Puerto Rico after translation
        pr_desired_location = (-75, 22)  # Adjust as needed
        delta_x = pr_desired_location[0] - pr_centroid.x
        delta_y = pr_desired_location[1] - pr_centroid.y

        # Apply translation
        puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print("Puerto Rico data is not available.")
        # Create an empty GeoDataFrame to prevent errors during concatenation
        puerto_rico_gdf = gpd.GeoDataFrame(columns=gdf.columns)

    # Combine all regions back into a single GeoDataFrame
    combined_gdf = pd.concat(
        [mainland_gdf, alaska_gdf, hawaii_gdf, puerto_rico_gdf], ignore_index=True
    )

    # Remove any rows with missing kg_per_mwh
    combined_gdf = combined_gdf.dropna(subset=['kg_per_mwh'])

    return combined_gdf

# Process data for selected years and store GeoDataFrames
years_to_process = [2019, 2020, 2021, 2022]
processed_data = {}
for year in years_to_process:
    if year in egrid_urls:
        gdf = process_year(year, egrid_urls[year])
        if gdf is not None:
            processed_data[year] = gdf
    else:
        print(f"No data available for year {year}.")

# Now plot subplots
num_years = len(processed_data)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Define common vmax and vmin for consistent color scaling
vmin = min(gdf['kg_per_mwh'].min() for gdf in processed_data.values())
vmax = max(gdf['kg_per_mwh'].max() for gdf in processed_data.values())

for ax, (year, combined_gdf) in zip(axes, sorted(processed_data.items())):
    combined_gdf.plot(
        column='kg_per_mwh',
        cmap='YlGnBu',
        linewidth=0.8,
        ax=ax,
        edgecolor='0.8',
        vmax=vmax,
        vmin=vmin
    )

    # Add labels for each subregion
    combined_gdf['coords'] = combined_gdf['geometry'].apply(lambda x: x.representative_point().coords[0])
    for idx, row in combined_gdf.iterrows():
        x, y = row['coords']
        # Manually adjust positions for overlapping labels
        if row['Name'] == 'NYLI':
            x_offset = 0.9
            y_offset = -0.9
        elif row['Name'] == 'NYCW':
            x_offset = 0.5
            y_offset = 1
        elif row['Name'] == 'PRMS':
            x_offset = 0
            y_offset = 0.7
        else:
            x_offset = 0
            y_offset = 0
        ax.annotate(
            text=row['Name'],
            xy=(x + x_offset, y + y_offset),
            horizontalalignment='center',
            fontsize=8,
            color='white',
            path_effects=[pe.withStroke(linewidth=2, foreground="black")]
        )

    # Adjust plot limits to include the new regions
    ax.set_xlim(-130, -65)
    ax.set_ylim(20, 50)

    # Simplify the title to display only the year
    ax.set_title(f"{year}", fontsize=15)

    ax.axis('off')


# Adjust layout and add a colorbar
plt.tight_layout()
fig.subplots_adjust(right=0.85, hspace=0.05)  # Set hspace close to zero for minimal vertical spacing

# Additional adjustment to overlap rows if needed
# Manually adjust each axis position to achieve a slight negative spacing effect
for i, ax in enumerate(axes):
    pos = ax.get_position()
    if i >= 2:  # Only move the bottom row up
        ax.set_position([pos.x0, pos.y0 + 0.12, pos.width, pos.height])  # Adjust as needed to control overlap

cbar_ax = fig.add_axes([0.88, 0.24, 0.03, 0.7])  # Move up by increasing the bottom and adjust height as needed

sm = plt.cm.ScalarMappable(cmap='YlGnBu', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Empty array for the scalar mappable
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('kg CO₂e/MWh', fontsize=12)

plt.savefig("egrid_carbon_intensity_subplots.png", dpi=300, bbox_inches='tight')
plt.savefig("egrid_carbon_intensity_subplots.pdf", dpi=150, bbox_inches='tight')

plt.show()


In [ ]:
import os
import requests
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.affinity import scale, translate
import matplotlib.patheffects as pe

# Directory to store downloaded data
local_data_dir = "./data/"
os.makedirs(local_data_dir, exist_ok=True)

# URLs for the 2014 shapefile and data file
data_file_url = "https://www.epa.gov/system/files/other-files/2023-01/egrid_historical_files_1996-2016.zip"
shapefile_url = "https://www.epa.gov/sites/default/files/2017-01/egrid_subregions.zip"

# Paths for downloaded ZIP files
data_file_zip_path = os.path.join(local_data_dir, "egrid_historical_files_1996-2016.zip")
shapefile_zip_path = os.path.join(local_data_dir, "egrid_subregions.zip")

# Directory paths for extracted files
data_file_dir = os.path.join(local_data_dir, "egrid_historical_files_1996-2016/egrid_historical_files_1996-2016")  # Updated to inner directory
shapefile_dir = os.path.join(local_data_dir, "egrid_subregions")

# Download function
def download_file(url, local_path):
    if not os.path.exists(local_path):
        print(f"Downloading from {url}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded to {local_path}.")
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
    else:
        print(f"{local_path} already downloaded.")

# Extract function
def extract_zip(zip_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to, exist_ok=True)
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted {zip_path} to {extract_to}.")
    else:
        print(f"{extract_to} already exists. Extraction skipped.")

# Download and extract data files
download_file(data_file_url, data_file_zip_path)
extract_zip(data_file_zip_path, os.path.join(local_data_dir, "egrid_historical_files_1996-2016"))

# Download and extract shapefile
download_file(shapefile_url, shapefile_zip_path)
extract_zip(shapefile_zip_path, shapefile_dir)

# Path to the 2014 data file
data_file_path = os.path.join(data_file_dir, "eGRID2014_Data_v2.xlsx")

# Load carbon intensity data from the Excel file
if not os.path.exists(data_file_path):
    print(f"Data file {data_file_path} not found. Available files in the directory:")
    for root, dirs, files in os.walk(data_file_dir):
        for file in files:
            print(os.path.join(root, file))
else:
    # Read the specific worksheet and columns
    intensity_df = pd.read_excel(data_file_path, sheet_name="SRL14", usecols=["SUBRGN", "SRC2ERTA"], skiprows=1)
    intensity_df["SRC2ERTA"] = pd.to_numeric(intensity_df["SRC2ERTA"], errors="coerce") * 0.45359237  # Convert lbs to kg
    intensity_df = intensity_df.dropna(subset=["SRC2ERTA"])

    print("Sample of carbon intensity data:")
    print(intensity_df.head())


    # Load the 2014 shapefile
    shapefile_path = os.path.join(shapefile_dir, "eGRID2014_subregions.shp")
    if not os.path.exists(shapefile_path):
        print(f"Shapefile {shapefile_path} not found.")
    else:
        gdf = gpd.read_file(shapefile_path)

        # Print available columns to identify the correct subregion code column
        print("Columns in shapefile:", gdf.columns)

        # Ensure geometries are in WGS84 CRS (EPSG:4326)
        if gdf.crs != 'EPSG:4326':
            print("Reprojecting geometries to WGS84 (EPSG:4326).")
            gdf = gdf.to_crs(epsg=4326)

        # Assuming 'SUBRGN' is the correct column name based on printed columns
        # Adjust the variable name if needed
        subregion_column = 'SUBRGN'  # Replace 'SUBRGN' if another name is used for subregions

        # Print a sample of the 'zips_for_G' column to inspect its contents
        print("Sample values in 'zips_for_G':")
        print(gdf['zips_for_G'].head())

        # If 'zips_for_G' contains subregion codes, use it as the subregion column for merging
        subregion_column = 'zips_for_G'  # Update this if necessary after inspecting the values

        # Merge shapefile with carbon intensity data
        gdf = gdf.merge(intensity_df, left_on=subregion_column, right_on="SUBRGN", how="left")

        # The rest of the processing and plotting logic follows...

        # Define subregions for Alaska, Hawaii, and Puerto Rico
        alaska_subregions = ['AKGD', 'AKMS']
        hawaii_subregions = ['HIMS', 'HIOA', 'HIMS/HIOA']
        puerto_rico_subregions = ['PR', 'PRVI', 'PRMS']

        # Separate regions for specific adjustments
        mainland_gdf = gdf[~gdf[subregion_column].isin(alaska_subregions + hawaii_subregions + puerto_rico_subregions)]
        alaska_gdf = gdf[gdf[subregion_column].isin(alaska_subregions)]
        hawaii_gdf = gdf[gdf[subregion_column].isin(hawaii_subregions)]
        puerto_rico_gdf = gdf[gdf[subregion_column].isin(puerto_rico_subregions)]

        # Alaska: Scale and translate
        if not alaska_gdf.empty:
            alaska_centroid = alaska_gdf.geometry.unary_union.centroid
            alaska_desired_location = (-120, 27)
            alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
                lambda geom: translate(scale(geom, xfact=0.41, yfact=0.5, origin=alaska_centroid),
                                    xoff=alaska_desired_location[0] - alaska_centroid.x,
                                    yoff=alaska_desired_location[1] - alaska_centroid.y)
            )

        # Hawaii: Scale and translate
        if not hawaii_gdf.empty:
            hawaii_centroid = hawaii_gdf.geometry.unary_union.centroid
            hawaii_desired_location = (-105, 22)
            hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
                lambda geom: translate(scale(geom, xfact=1.5, yfact=1.5, origin=hawaii_centroid),
                                    xoff=hawaii_desired_location[0] - hawaii_centroid.x,
                                    yoff=hawaii_desired_location[1] - hawaii_centroid.y)
            )

        # Puerto Rico: Scale and translate
        if not puerto_rico_gdf.empty:
            pr_centroid = puerto_rico_gdf.geometry.unary_union.centroid
            pr_desired_location = (-75, 22)
            puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
                lambda geom: translate(scale(geom, xfact=1.5, yfact=1.5, origin=pr_centroid),
                                    xoff=pr_desired_location[0] - pr_centroid.x,
                                    yoff=pr_desired_location[1] - pr_centroid.y)
            )

        # Combine all regions back into a single GeoDataFrame
        combined_gdf = pd.concat([mainland_gdf, alaska_gdf, hawaii_gdf, puerto_rico_gdf], ignore_index=True)

        # Plot
        fig, ax = plt.subplots(figsize=(12, 8))
        combined_gdf.plot(column='SRC2ERTA', cmap='YlGnBu', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

        # Add labels for each subregion
        combined_gdf['coords'] = combined_gdf['geometry'].apply(lambda x: x.representative_point().coords[0])
        for idx, row in combined_gdf.iterrows():
            x, y = row['coords']
            ax.annotate(
                text=row[subregion_column],
                xy=(x, y),
                horizontalalignment='center',
                fontsize=8,
                color='white',
                path_effects=[pe.withStroke(linewidth=2, foreground="black")]
            )

        # Customize plot appearance
        # Adjust plot limits to include the new regions
        ax.set_xlim(-130, -65)
        ax.set_ylim(20, 50)

        ax.set_title("eGRID Subregions - 2014 CO₂ Intensity (kg/MWh)", fontsize=15)
        ax.axis('off')
        plt.tight_layout()
        plt.show()


In [3]:
# gdf = extract_to_geojson(kml_path, year)
# print(gdf.head())  # Debugging line



In [4]:
# print(gdf['Description'].head(10))  # Print the first 10 descriptions to see what they look like


# new 2014

In [ ]:
import os
import requests
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.affinity import scale, translate
import matplotlib.patheffects as pe

# Directory to store downloaded data
local_data_dir = "./data/"
os.makedirs(local_data_dir, exist_ok=True)

# eGRID Data URLs
egrid_urls = {
    2014: {
        "data_zip": "https://www.epa.gov/system/files/other-files/2023-01/egrid_historical_files_1996-2016.zip",
        "shapefile_zip": "https://www.epa.gov/sites/default/files/2017-01/egrid_subregions.zip",
    },
    2019: {
        "egrid_xlsx": 'https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx',
        "subregions_kmz": 'https://www.epa.gov/sites/default/files/2021-02/egrid2019_subregions.kmz',
    },
    2021: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2023-01/eGRID2021_data.xlsx',
        "subregions_shp": 'https://gaftp.epa.gov/EPADataCommons/ORD/egrid2021/eGRID2021%20Subregions%20Shapefile.zip',
    },
    2022: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2023-01/egrid2022_data.xlsx',
        "subregions_kmz": 'https://www.epa.gov/system/files/other-files/2023-01/egrid2022_subregions.kmz',
    },
}

# Conversion factor from pounds to kilograms
LBS_PER_KG = 0.45359237

# Function to download files
def download_file(url, local_path):
    if not os.path.exists(local_path):
        print(f"Downloading from {url}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded to {local_path}.")
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
    else:
        print(f"{local_path} already downloaded.")

# Function to extract ZIP files
def extract_zip(zip_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to, exist_ok=True)
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted {zip_path} to {extract_to}.")
    else:
        print(f"{extract_to} already exists. Extraction skipped.")

# Function to process data for 2014
def process_2014_data(urls):
    print("Processing 2014 data with specialized logic...")

    # URLs for the 2014 shapefile and data file
    data_file_url = urls["data_zip"]
    shapefile_url = urls["shapefile_zip"]

    # Paths for downloaded ZIP files
    data_file_zip_path = os.path.join(local_data_dir, "egrid_historical_files_1996-2016.zip")
    shapefile_zip_path = os.path.join(local_data_dir, "egrid_subregions.zip")

    # Directory paths for extracted files
    data_file_dir = os.path.join(local_data_dir, "egrid_historical_files_1996-2016")
    shapefile_dir = os.path.join(local_data_dir, "egrid_subregions")

    # Download and extract data files
    download_file(data_file_url, data_file_zip_path)
    extract_zip(data_file_zip_path, data_file_dir)

    # Download and extract shapefile
    download_file(shapefile_url, shapefile_zip_path)
    extract_zip(shapefile_zip_path, shapefile_dir)

    # Adjust data_file_dir to account for nested directories
    nested_data_dir = os.path.join(data_file_dir, "egrid_historical_files_1996-2016")
    if os.path.exists(nested_data_dir):
        data_file_dir = nested_data_dir

    # Path to the 2014 data file
    data_file_path = os.path.join(data_file_dir, "eGRID2014_Data_v2.xlsx")

    # Verify that the data file exists
    if not os.path.exists(data_file_path):
        print(f"Data file {data_file_path} not found. Available files in the directory:")
        for root, dirs, files in os.walk(data_file_dir):
            for file in files:
                print(os.path.join(root, file))
        return None

    # Load carbon intensity data from the Excel file
    try:
        # Read the specific worksheet and columns
        intensity_df = pd.read_excel(data_file_path, sheet_name="SRL14", usecols=["SUBRGN", "SRC2ERTA"], skiprows=1)
        intensity_df["SRC2ERTA"] = pd.to_numeric(intensity_df["SRC2ERTA"], errors="coerce")
        intensity_df = intensity_df.dropna(subset=["SRC2ERTA"])

        # Convert lbs to kg
        intensity_df["kg_per_mwh"] = intensity_df["SRC2ERTA"] * LBS_PER_KG

        print("Sample of carbon intensity data:")
        print(intensity_df.head())

    except Exception as e:
        print(f"Failed to read eGRID data for 2014: {e}")
        return None

    # Load the 2014 shapefile
    try:
        shapefile_path = os.path.join(shapefile_dir, "eGRID2014_subregions.shp")
        gdf = gpd.read_file(shapefile_path)

        # Determine the subregion column
        if 'SUBRGN' in gdf.columns:
            subregion_column = 'SUBRGN'
        elif 'zips_for_G' in gdf.columns:
            subregion_column = 'zips_for_G'
        else:
            print("Subregion column not found in shapefile.")
            return None

        # Ensure geometries are in WGS84 CRS (EPSG:4326)
        if gdf.crs != 'EPSG:4326':
            print("Reprojecting geometries to WGS84 (EPSG:4326).")
            gdf = gdf.to_crs(epsg=4326)

        # Merge shapefile with carbon intensity data
        gdf = gdf.merge(intensity_df[['SUBRGN', 'kg_per_mwh']], left_on=subregion_column, right_on="SUBRGN", how="left")

        # Rename the subregion column to 'Name' to match other years
        gdf = gdf.rename(columns={subregion_column: 'Name'})

    except Exception as e:
        print(f"Failed to read subregions for 2014: {e}")
        return None

    # Adjust regions and return the processed GeoDataFrame
    combined_gdf = adjust_regions(gdf, 2014)
    return combined_gdf

# Function to process data for other years
def process_year(year, urls):
    print(f"\nProcessing data for year {year}...")

    if year == 2014:
        return process_2014_data(urls)
    else:
        # Paths for Excel and subregions files
        egrid_xlsx_url = urls['egrid_xlsx']
        egrid_xlsx_path = os.path.join(local_data_dir, f"egrid{year}_data.xlsx")

        # Determine the subregions file type and path
        if 'subregions_kml' in urls:
            subregions_url = urls['subregions_kml']
            subregions_ext = 'kml'
        elif 'subregions_kmz' in urls:
            subregions_url = urls['subregions_kmz']
            subregions_ext = 'kmz'
        elif 'subregions_shp' in urls:
            subregions_url = urls['subregions_shp']
            subregions_ext = 'zip'
        else:
            print(f"No subregions file provided for year {year}. Skipping...")
            return None

        subregions_path = os.path.join(local_data_dir, f"egrid{year}_subregions.{subregions_ext}")

        # Download files
        download_file(egrid_xlsx_url, egrid_xlsx_path)
        download_file(subregions_url, subregions_path)

        # Load carbon intensity data
        try:
            sheet_name = f'SRL{str(year)[-2:]}'
            egrid_intensity_df = pd.read_excel(egrid_xlsx_path, sheet_name=sheet_name)

            # Use the relevant column for CO₂ intensity
            co2_column_name = [col for col in egrid_intensity_df.columns if 'CO2 total output emission rate (lb/MWh)' in col]
            if co2_column_name:
                co2_column_name = co2_column_name[0]
            else:
                raise ValueError("CO₂ emission rate column not found.")

            # Extract subregion acronym and carbon intensity
            region_intensity = egrid_intensity_df[['eGRID subregion acronym', co2_column_name]].copy()
            region_intensity.columns = ['SUBRGN', 'SRC2ERTA']

            # Convert to numeric and handle NaNs
            region_intensity['SRC2ERTA'] = pd.to_numeric(region_intensity['SRC2ERTA'], errors='coerce')
            region_intensity = region_intensity.dropna(subset=['SRC2ERTA'])

            # Convert lbs to kg
            region_intensity['kg_per_mwh'] = region_intensity['SRC2ERTA'] * LBS_PER_KG
            region_intensity = region_intensity[['SUBRGN', 'kg_per_mwh']]
            region_intensity['SUBRGN'] = region_intensity['SUBRGN'].astype(str)

            print(f"Extracted intensity data for {year}:")
            print(region_intensity.head())

        except Exception as e:
            print(f"Failed to read eGRID data for {year}: {e}")
            return None

        # Load subregions into GeoDataFrame
        try:
            if subregions_ext == 'kmz':
                with ZipFile(subregions_path, 'r') as kmz:
                    kml_filename = [name for name in kmz.namelist() if name.endswith('.kml')][0]
                    kmz.extract(kml_filename, local_data_dir)
                    kml_path = os.path.join(local_data_dir, kml_filename)
                    gdf = gpd.read_file(kml_path, driver='KML')
                    os.remove(kml_path)
            elif subregions_ext == 'kml':
                gdf = gpd.read_file(subregions_path, driver='KML')
            elif subregions_ext == 'zip':
                # Extract the zip file
                with ZipFile(subregions_path, 'r') as zip_ref:
                    zip_ref.extractall(local_data_dir)
                    # Find the shapefile (.shp)
                    shapefile_name = [name for name in zip_ref.namelist() if name.endswith('.shp')][0]
                    shapefile_path = os.path.join(local_data_dir, shapefile_name)
                gdf = gpd.read_file(shapefile_path)
            else:
                print(f"Unsupported subregions file format for year {year}.")
                return None

            # Check for 'SUBRGN' or 'Name' column and rename to 'Name'
            if 'SUBRGN' in gdf.columns:
                gdf = gdf.rename(columns={'SUBRGN': 'Name'})
            elif 'Name' not in gdf.columns:
                raise ValueError("Subregion name column not found in geospatial data.")

            # Ensure geometries are in WGS84 CRS (EPSG:4326)
            if gdf.crs != 'EPSG:4326':
                print(f"Reprojecting geometries to WGS84 for year {year}.")
                gdf = gdf.to_crs(epsg=4326)

            # Merge with carbon intensity data using 'Name' as the identifier
            gdf = gdf.merge(region_intensity, left_on='Name', right_on='SUBRGN', how='left')

        except Exception as e:
            print(f"Failed to read subregions for {year}: {e}")
            return None

        # Adjust regions and return the processed GeoDataFrame
        combined_gdf = adjust_regions(gdf, year)
        return combined_gdf

# Function to adjust regions (without plotting)
def adjust_regions(gdf, year):
    # Define subregions for Alaska and Hawaii
    alaska_subregions = ['AKGD', 'AKMS']
    hawaii_subregions = ['HIMS', 'HIOA', 'HIMS/HIOA']
    puerto_rico_subregions = ['PR', 'PRVI', 'PRMS']

    # Attempt to find Puerto Rico subregions in the data
    pr_subregions = gdf[gdf['Name'].str.contains('PR', case=False, na=False)]['Name'].unique()
    if len(pr_subregions) == 0:
        # Try to identify PR subregions differently
        pr_subregions = [name for name in gdf['Name'].unique() if 'PR' in str(name).upper()]
    print(f"Puerto Rico subregions in data for {year}: {pr_subregions}")

    # Separate the regions
    mainland_gdf = gdf[~gdf['Name'].isin(alaska_subregions + hawaii_subregions + list(pr_subregions))]
    alaska_gdf = gdf[gdf['Name'].isin(alaska_subregions)]
    hawaii_gdf = gdf[gdf['Name'].isin(hawaii_subregions)]
    puerto_rico_gdf = gdf[gdf['Name'].isin(pr_subregions)]

    # Alaska: Scale and translate
    if not alaska_gdf.empty:
        alaska_gdf = alaska_gdf.copy()

        # Compute the centroid of Alaska (combined geometries)
        alaska_centroid = alaska_gdf.geometry.unary_union.centroid

        # Desired location for Alaska after translation
        alaska_desired_location = (-120, 27)  # Adjust as needed

        # Apply scaling with the common origin (alaska_centroid)
        alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=0.41, yfact=0.5, origin=alaska_centroid)
        )

        # After scaling, compute the new centroid
        scaled_alaska_centroid = alaska_gdf.geometry.unary_union.centroid

        # Compute translation amounts based on the desired location
        delta_x = alaska_desired_location[0] - scaled_alaska_centroid.x
        delta_y = alaska_desired_location[1] - scaled_alaska_centroid.y

        # Apply translation to each geometry
        alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print(f"Alaska data is not available for {year}.")

    # Hawaii: Scale and translate
    if not hawaii_gdf.empty:
        hawaii_gdf = hawaii_gdf.copy()
        hawaii_centroid = hawaii_gdf.geometry.unary_union.centroid

        # Apply scaling
        hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=1.5, yfact=1.5, origin=hawaii_centroid)
        )

        # Desired location for Hawaii after translation
        hawaii_desired_location = (-105, 22)  # Adjust as needed
        delta_x = hawaii_desired_location[0] - hawaii_centroid.x
        delta_y = hawaii_desired_location[1] - hawaii_centroid.y

        # Apply translation
        hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print(f"Hawaii data is not available for {year}.")

    # Puerto Rico: Scale and translate
    if not puerto_rico_gdf.empty:
        puerto_rico_gdf = puerto_rico_gdf.copy()
        pr_centroid = puerto_rico_gdf.geometry.unary_union.centroid

        # Apply scaling
        puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=1.5, yfact=1.5, origin=pr_centroid)
        )

        # Desired location for Puerto Rico after translation
        pr_desired_location = (-75, 22)  # Adjust as needed
        delta_x = pr_desired_location[0] - pr_centroid.x
        delta_y = pr_desired_location[1] - pr_centroid.y

        # Apply translation
        puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print(f"Puerto Rico data is not available for {year}.")

    # Combine all regions back into a single GeoDataFrame
    combined_gdf = pd.concat(
        [mainland_gdf, alaska_gdf, hawaii_gdf, puerto_rico_gdf], ignore_index=True
    )

    # Remove any rows with missing kg_per_mwh
    combined_gdf = combined_gdf.dropna(subset=['kg_per_mwh'])

    return combined_gdf

# Process data for selected years and store GeoDataFrames
years_to_process = [2014, 2019, 2021, 2022]
processed_data = {}
for year in years_to_process:
    if year in egrid_urls:
        gdf = process_year(year, egrid_urls[year])
        if gdf is not None:
            processed_data[year] = gdf
    else:
        print(f"No data available for year {year}.")

# Now plot subplots
num_years = len(processed_data)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Define common vmax and vmin for consistent color scaling
vmin = min(gdf['kg_per_mwh'].min() for gdf in processed_data.values())
vmax = max(gdf['kg_per_mwh'].max() for gdf in processed_data.values())

for ax, (year, combined_gdf) in zip(axes, sorted(processed_data.items())):
    combined_gdf.plot(
        column='kg_per_mwh',
        cmap='YlGnBu',
        linewidth=0.8,
        ax=ax,
        edgecolor='0.8',
        vmax=vmax,
        vmin=vmin
    )

    # Add labels for each subregion
    combined_gdf['coords'] = combined_gdf['geometry'].apply(lambda x: x.representative_point().coords[0])
    for idx, row in combined_gdf.iterrows():
        x, y = row['coords']
        # Manually adjust positions for overlapping labels
        if row['Name'] == 'NYLI':
            x_offset = 0.9
            y_offset = -0.9
        elif row['Name'] == 'NYCW':
            x_offset = 0.5
            y_offset = 1
        elif row['Name'] == 'PRMS':
            x_offset = 0
            y_offset = 0.7
        else:
            x_offset = 0
            y_offset = 0
        ax.annotate(
            text=row['Name'],
            xy=(x + x_offset, y + y_offset),
            horizontalalignment='center',
            fontsize=8,
            color='white',
            path_effects=[pe.withStroke(linewidth=2, foreground="black")]
        )

    # Adjust plot limits to include the new regions
    ax.set_xlim(-130, -65)
    ax.set_ylim(20, 50)

    # Simplify the title to display only the year
    ax.set_title(f"{year}", fontsize=15)

    ax.axis('off')

# Adjust layout and add a colorbar
plt.tight_layout()
fig.subplots_adjust(right=0.85, hspace=0.05)  # Set hspace close to zero for minimal vertical spacing

# Additional adjustment to overlap rows if needed
# Manually adjust each axis position to achieve a slight negative spacing effect
for i, ax in enumerate(axes):
    pos = ax.get_position()
    if i >= 2:  # Only move the bottom row up
        ax.set_position([pos.x0, pos.y0 + 0.12, pos.width, pos.height])  # Adjust as needed

cbar_ax = fig.add_axes([0.88, 0.24, 0.03, 0.7])  # Adjust as needed

sm = plt.cm.ScalarMappable(cmap='YlGnBu', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Empty array for the scalar mappable
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('kg CO₂e/MWh', fontsize=12)

plt.savefig("egrid_carbon_intensity_subplots.png", dpi=300, bbox_inches='tight')
plt.savefig("egrid_carbon_intensity_subplots.pdf", dpi=150, bbox_inches='tight')

plt.show()


# Delta

In [ ]:
import os
import requests
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.affinity import scale, translate
import matplotlib.patheffects as pe
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Directory to store downloaded data
local_data_dir = "./data/"
os.makedirs(local_data_dir, exist_ok=True)

# eGRID Data URLs
egrid_urls = {
    2014: {
        "data_zip": "https://www.epa.gov/system/files/other-files/2023-01/egrid_historical_files_1996-2016.zip",
        "shapefile_zip": "https://www.epa.gov/sites/default/files/2017-01/egrid_subregions.zip",
    },
    2020: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2022-09/eGRID2020_Data_v2.xlsx',
        "subregions_kmz": 'https://www.epa.gov/system/files/other-files/2022-01/egrid2020_subregions.kmz',
    },
    2022: {
        "egrid_xlsx": 'https://www.epa.gov/system/files/documents/2023-01/egrid2022_data.xlsx',
        "subregions_kmz": 'https://www.epa.gov/system/files/other-files/2023-01/egrid2022_subregions.kmz',
    },
}

# Conversion factor from pounds to kilograms
LBS_PER_KG = 0.45359237

# Function to download files
def download_file(url, local_path):
    if not os.path.exists(local_path):
        print(f"Downloading from {url}...")
        response = requests.get(url, verify=False)  # Added verify=False to handle SSL issues
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded to {local_path}.")
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
    else:
        print(f"{local_path} already downloaded.")

# Function to extract ZIP files
def extract_zip(zip_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to, exist_ok=True)
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted {zip_path} to {extract_to}.")
    else:
        print(f"{extract_to} already exists. Extraction skipped.")

# Function to process data for 2014
def process_2014_data(urls):
    print("Processing 2014 data with specialized logic...")

    # URLs for the 2014 shapefile and data file
    data_file_url = urls["data_zip"]
    shapefile_url = urls["shapefile_zip"]

    # Paths for downloaded ZIP files
    data_file_zip_path = os.path.join(local_data_dir, "egrid_historical_files_1996-2016.zip")
    shapefile_zip_path = os.path.join(local_data_dir, "egrid_subregions_2014.zip")

    # Directory paths for extracted files
    data_file_dir = os.path.join(local_data_dir, "egrid_historical_files_1996-2016")
    shapefile_dir = os.path.join(local_data_dir, "egrid_subregions_2014")

    # Download and extract data files
    download_file(data_file_url, data_file_zip_path)
    extract_zip(data_file_zip_path, data_file_dir)

    # Download and extract shapefile
    download_file(shapefile_url, shapefile_zip_path)
    extract_zip(shapefile_zip_path, shapefile_dir)

    # Adjust data_file_dir to account for nested directories
    nested_data_dir = os.path.join(data_file_dir, "egrid_historical_files_1996-2016")
    if os.path.exists(nested_data_dir):
        data_file_dir = nested_data_dir

    # Path to the 2014 data file
    data_file_path = os.path.join(data_file_dir, "eGRID2014_Data_v2.xlsx")

    # Verify that the data file exists
    if not os.path.exists(data_file_path):
        print(f"Data file {data_file_path} not found.")
        return None

    # Load carbon intensity data from the Excel file
    try:
        # Read the specific worksheet and columns
        intensity_df = pd.read_excel(data_file_path, sheet_name="SRL14", usecols=["SUBRGN", "SRC2ERTA"], skiprows=1)
        intensity_df["SRC2ERTA"] = pd.to_numeric(intensity_df["SRC2ERTA"], errors="coerce")
        intensity_df = intensity_df.dropna(subset=["SRC2ERTA"])

        # Convert lbs to kg
        intensity_df["kg_per_mwh"] = intensity_df["SRC2ERTA"] * LBS_PER_KG

        # Keep only the required columns
        intensity_df = intensity_df[['SUBRGN', 'kg_per_mwh']]

        # Remove Puerto Rico (not available in 2014)
        intensity_df = intensity_df[~intensity_df['SUBRGN'].str.contains('PR', na=False)]

        print("Sample of carbon intensity data for 2014:")
        print(intensity_df.head())

    except Exception as e:
        print(f"Failed to read eGRID data for 2014: {e}")
        return None

    return intensity_df

# Function to process data for a given year (2020 or 2022)
def process_year_data(year, urls, include_puerto_rico=True):
    print(f"Processing {year} data...")

    # Paths for Excel and subregions files
    egrid_xlsx_url = urls['egrid_xlsx']
    egrid_xlsx_path = os.path.join(local_data_dir, f"egrid{year}_data.xlsx")

    subregions_url = urls['subregions_kmz']
    subregions_path = os.path.join(local_data_dir, f"egrid{year}_subregions.kmz")

    # Download files
    download_file(egrid_xlsx_url, egrid_xlsx_path)
    download_file(subregions_url, subregions_path)

    # Load carbon intensity data
    try:
        sheet_name = f'SRL{str(year)[-2:]}'
        egrid_intensity_df = pd.read_excel(egrid_xlsx_path, sheet_name=sheet_name)

        # Use the relevant column for CO₂ intensity
        co2_column_name = [col for col in egrid_intensity_df.columns if 'CO2 total output emission rate (lb/MWh)' in col]
        if co2_column_name:
            co2_column_name = co2_column_name[0]
        else:
            raise ValueError("CO₂ emission rate column not found.")

        # Extract subregion acronym and carbon intensity
        region_intensity = egrid_intensity_df[['eGRID subregion acronym', co2_column_name]].copy()
        region_intensity.columns = ['SUBRGN', 'SRC2ERTA']

        # Convert to numeric and handle NaNs
        region_intensity['SRC2ERTA'] = pd.to_numeric(region_intensity['SRC2ERTA'], errors='coerce')
        region_intensity = region_intensity.dropna(subset=['SRC2ERTA'])

        # Convert lbs to kg
        region_intensity['kg_per_mwh'] = region_intensity['SRC2ERTA'] * LBS_PER_KG
        region_intensity = region_intensity[['SUBRGN', 'kg_per_mwh']]
        region_intensity['SUBRGN'] = region_intensity['SUBRGN'].astype(str)

        # Optionally remove Puerto Rico
        if not include_puerto_rico:
            region_intensity = region_intensity[~region_intensity['SUBRGN'].str.contains('PR', na=False)]

        print(f"Sample of carbon intensity data for {year}:")
        print(region_intensity.head())

    except Exception as e:
        print(f"Failed to read eGRID data for {year}: {e}")
        return None, None

    # Load subregions into GeoDataFrame
    try:
        # Extract the KML file from the KMZ
        with ZipFile(subregions_path, 'r') as kmz:
            kml_filename = [name for name in kmz.namelist() if name.endswith('.kml')][0]
            kmz.extract(kml_filename, local_data_dir)
            kml_path = os.path.join(local_data_dir, kml_filename)
            gdf = gpd.read_file(kml_path, driver='KML')
            os.remove(kml_path)

        # Check for 'Name' column
        if 'Name' not in gdf.columns:
            raise ValueError("Subregion name column not found in geospatial data.")

        # Optionally remove Puerto Rico from the geodataframe
        if not include_puerto_rico:
            gdf = gdf[~gdf['Name'].str.contains('PR', na=False)]

        # Ensure geometries are in WGS84 CRS (EPSG:4326)
        if gdf.crs != 'EPSG:4326':
            print(f"Reprojecting geometries to WGS84 for {year}.")
            gdf = gdf.to_crs(epsg=4326)

        print(f"Subregions in {year} geodataframe: {gdf['Name'].unique()}")

    except Exception as e:
        print(f"Failed to read subregions for {year}: {e}")
        return None, None

    return region_intensity, gdf

# Adjust regions (Alaska, Hawaii, and Puerto Rico)
def adjust_regions_delta(gdf):
    # Define subregions for Alaska, Hawaii, and Puerto Rico
    alaska_subregions = ['AKGD', 'AKMS']
    hawaii_subregions = ['HIMS', 'HIOA', 'HIMS/HIOA']
    puerto_rico_subregions = ['PR', 'PRMS', 'PRMSA']

    # Separate the regions
    mainland_gdf = gdf[~gdf['Name'].isin(alaska_subregions + hawaii_subregions + puerto_rico_subregions)]
    alaska_gdf = gdf[gdf['Name'].isin(alaska_subregions)]
    hawaii_gdf = gdf[gdf['Name'].isin(hawaii_subregions)]
    puerto_rico_gdf = gdf[gdf['Name'].isin(puerto_rico_subregions)]

    # Alaska: Scale and translate
    if not alaska_gdf.empty:
        alaska_gdf = alaska_gdf.copy()

        # Compute the centroid of Alaska (combined geometries)
        alaska_centroid = alaska_gdf.geometry.unary_union.centroid

        # Desired location for Alaska after translation
        alaska_desired_location = (-120, 27)  # Adjust as needed

        # Apply scaling with the common origin (alaska_centroid)
        alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=0.41, yfact=0.5, origin=alaska_centroid)
        )

        # After scaling, compute the new centroid
        scaled_alaska_centroid = alaska_gdf.geometry.unary_union.centroid

        # Compute translation amounts based on the desired location
        delta_x = alaska_desired_location[0] - scaled_alaska_centroid.x
        delta_y = alaska_desired_location[1] - scaled_alaska_centroid.y

        # Apply translation to each geometry
        alaska_gdf['geometry'] = alaska_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print("Alaska data is not available.")

    # Hawaii: Scale and translate
    if not hawaii_gdf.empty:
        hawaii_gdf = hawaii_gdf.copy()
        hawaii_centroid = hawaii_gdf.geometry.unary_union.centroid

        # Apply scaling
        hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=1.5, yfact=1.5, origin=hawaii_centroid)
        )

        # Desired location for Hawaii after translation
        hawaii_desired_location = (-105, 22)  # Adjust as needed
        delta_x = hawaii_desired_location[0] - hawaii_centroid.x
        delta_y = hawaii_desired_location[1] - hawaii_centroid.y

        # Apply translation
        hawaii_gdf['geometry'] = hawaii_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print("Hawaii data is not available.")

    # Puerto Rico: Scale and translate
    if not puerto_rico_gdf.empty:
        puerto_rico_gdf = puerto_rico_gdf.copy()
        pr_centroid = puerto_rico_gdf.geometry.unary_union.centroid

        # Apply scaling
        puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
            lambda geom: scale(geom, xfact=2.5, yfact=2.5, origin=pr_centroid)
        )

        # Desired location for Puerto Rico after translation
        pr_desired_location = (-75, 20)  # Adjust as needed
        delta_x = pr_desired_location[0] - pr_centroid.x
        delta_y = pr_desired_location[1] - pr_centroid.y

        # Apply translation
        puerto_rico_gdf['geometry'] = puerto_rico_gdf['geometry'].apply(
            lambda geom: translate(geom, xoff=delta_x, yoff=delta_y)
        )
    else:
        print("Puerto Rico data is not available.")

    # Combine all regions back into a single GeoDataFrame
    combined_gdf = pd.concat(
        [mainland_gdf, alaska_gdf, hawaii_gdf, puerto_rico_gdf], ignore_index=True
    )

    return combined_gdf

# Process data for 2014
intensity_df_2014 = process_2014_data(egrid_urls[2014])
if intensity_df_2014 is None:
    print("Failed to process 2014 data.")
    exit()

# Process data for 2020, excluding Puerto Rico for 2014-2020 delta
region_intensity_2020_no_pr, gdf_2020_no_pr = process_year_data(2020, egrid_urls[2020], include_puerto_rico=False)
if region_intensity_2020_no_pr is None or gdf_2020_no_pr is None:
    print("Failed to process 2020 data.")
    exit()

# Merge 2014 and 2020 data on 'SUBRGN'
merged_intensity_2014_2020 = pd.merge(intensity_df_2014, region_intensity_2020_no_pr, on='SUBRGN', suffixes=('_2014', '_2020'))

# Compute the delta
merged_intensity_2014_2020['delta_kg_per_mwh'] = merged_intensity_2014_2020['kg_per_mwh_2020'] - merged_intensity_2014_2020['kg_per_mwh_2014']

print("Merged intensity data with delta from 2014 to 2020:")
print(merged_intensity_2014_2020.head())

# Merge with the 2020 geodataframe
gdf_delta_2014_2020 = gdf_2020_no_pr.merge(merged_intensity_2014_2020[['SUBRGN', 'delta_kg_per_mwh']], left_on='Name', right_on='SUBRGN', how='left')

# Adjust regions
gdf_delta_2014_2020 = adjust_regions_delta(gdf_delta_2014_2020)

gdf_delta_2014_2020['geometry'] = gdf_delta_2014_2020['geometry'].simplify(tolerance=0.1)


# Plot delta from 2014 to 2020
fig, ax = plt.subplots(figsize=(12, 8))

# Define diverging colormap
cmap = plt.cm.RdBu_r

# Define vmax and vmin for color scaling
vmax = gdf_delta_2014_2020['delta_kg_per_mwh'].abs().max()
vmin = -vmax

gdf_delta_2014_2020.plot(
    column='delta_kg_per_mwh',
    cmap=cmap,
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    vmax=vmax,
    vmin=vmin
)

# Define a function to calculate brightness based on RGB values
def is_dark_color(value, cmap, vmin, vmax):
    """Returns True if the color is dark, False otherwise."""
    norm_value = (value - vmin) / (vmax - vmin)  # Normalize value to be between 0 and 1
    rgb = cmap(norm_value)[:3]  # Get RGB from colormap
    brightness = (0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2])  # Perceived brightness formula
    return brightness < 0.5  # Threshold to determine if color is "dark"

# Add labels for each subregion
gdf_delta_2014_2020['coords'] = gdf_delta_2014_2020['geometry'].apply(lambda x: x.representative_point().coords[0])
for idx, row in gdf_delta_2014_2020.iterrows():
    x, y = row['coords']
    # Manually adjust positions for overlapping labels
    if row['Name'] == 'NYLI':
        x_offset = 0.9
        y_offset = -0.9
    elif row['Name'] == 'NYCW':
        x_offset = 0.5
        y_offset = 1
    elif row['Name'] == 'PRMS':
        x_offset = 0
        y_offset = 0.4
    else:
        x_offset = 0
        y_offset = 0

    # Determine color based on brightness
    color = 'white' if is_dark_color(row['delta_kg_per_mwh'], cmap, vmin, vmax) else 'black'
    
    ax.annotate(
        text=row['Name'],
        xy=(x + x_offset, y + y_offset),
        horizontalalignment='center',
        fontsize=11,
        color=color,
        # path_effects=[pe.withStroke(linewidth=1.5, foreground="black")]
    )

# Customize plot appearance
ax.set_xlim(-130, -65)
ax.set_ylim(17, 50)

# ax.set_title("Change in eGRID Subregion CO₂ Intensity from 2014 to 2020 (kg/MWh)", fontsize=15)
ax.axis('off')

cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4]) 

# Create the colorbar using the custom axis
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm, cax=cbar_ax, pad=0.1)
cbar.set_label('Change in kg CO₂e/MWh', fontsize=12)

plt.tight_layout()
plt.savefig("egrid_delta_2014_2020.pdf", dpi=150)
plt.savefig("egrid_delta_2014_2020.png", dpi=300, transparent=True)
plt.show()

# Now process data for 2020 and 2022, including Puerto Rico
region_intensity_2020_with_pr, gdf_2020_with_pr = process_year_data(2020, egrid_urls[2020], include_puerto_rico=True)
region_intensity_2022, gdf_2022 = process_year_data(2022, egrid_urls[2022], include_puerto_rico=True)

if region_intensity_2020_with_pr is None or gdf_2020_with_pr is None or region_intensity_2022 is None or gdf_2022 is None:
    print("Failed to process 2020 or 2022 data.")
    exit()

# Merge 2020 and 2022 data on 'SUBRGN'
merged_intensity_2020_2022 = pd.merge(region_intensity_2020_with_pr, region_intensity_2022, on='SUBRGN', suffixes=('_2020', '_2022'))

# Compute the delta
merged_intensity_2020_2022['delta_kg_per_mwh'] = merged_intensity_2020_2022['kg_per_mwh_2022'] - merged_intensity_2020_2022['kg_per_mwh_2020']

print("Merged intensity data with delta from 2020 to 2022:")
print(merged_intensity_2020_2022.head())

# Merge with the 2022 geodataframe
gdf_delta_2020_2022 = gdf_2022.merge(merged_intensity_2020_2022[['SUBRGN', 'delta_kg_per_mwh']], left_on='Name', right_on='SUBRGN', how='left')

# Adjust regions
gdf_delta_2020_2022 = adjust_regions_delta(gdf_delta_2020_2022)

gdf_delta_2020_2022['geometry'] = gdf_delta_2020_2022['geometry'].simplify(tolerance=0.1)


# Plot delta from 2020 to 2022
fig, ax = plt.subplots(figsize=(12, 8))

# Define diverging colormap
cmap = plt.cm.RdBu_r

# Define vmax and vmin for color scaling
# vmax = gdf_delta_2020_2022['delta_kg_per_mwh'].abs().max()
vmax = 75
vmin = -75
# vmin = -vmax

gdf_delta_2020_2022.plot(
    column='delta_kg_per_mwh',
    cmap=cmap,
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    vmax=vmax,
    vmin=vmin
)

# Add labels for each subregion
gdf_delta_2020_2022['coords'] = gdf_delta_2020_2022['geometry'].apply(lambda x: x.representative_point().coords[0])
for idx, row in gdf_delta_2020_2022.iterrows():
    x, y = row['coords']
    # Manually adjust positions for overlapping labels
    if row['Name'] == 'NYLI':
        x_offset = 0.9
        y_offset = -0.9
    elif row['Name'] == 'NYCW':
        x_offset = 0.5
        y_offset = 1
    elif row['Name'] == 'PRMS':
        x_offset = 0
        y_offset = 0.4
    else:
        x_offset = 0
        y_offset = 0

        # Determine color based on brightness
    color = 'white' if is_dark_color(row['delta_kg_per_mwh'], cmap, vmin, vmax) else 'black'

    if row['Name'] == 'NYCW':
        color='black'
    ax.annotate(
        text=row['Name'],
        xy=(x + x_offset, y + y_offset),
        horizontalalignment='center',
        fontsize=13,
        color=color,
        # path_effects=[pe.withStroke(linewidth=1.5, foreground="black")]
    )

# Customize plot appearance
ax.set_xlim(-130, -65)
ax.set_ylim(17, 50)

# ax.set_title("Change in eGRID Subregion CO₂ Intensity from 2020 to 2022 (kg/MWh)", fontsize=15)
ax.axis('off')

cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4]) 

# Create the colorbar using the custom axis
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm, cax=cbar_ax, pad=0.1)
cbar.set_label('Change in kg CO₂e/MWh', fontsize=12)

plt.tight_layout()
plt.savefig("egrid_delta_2020_2022.pdf", dpi=150)
plt.savefig("egrid_delta_2020_2022.png", dpi=300, transparent=True)
plt.show()
